In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import openweathermapy.core as owm
import json

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [63]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
latitudes = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        latitudes.append(lats)

# Print the city count to confirm sufficient count
print(len(cities))

613


## Perform API Calls

In [64]:
# OpenWeatherMap API Key
api_key = api_keys.api_key
settings = {"units": "imperial", "appid": api_key}

# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="

query_url = url + api_key + "&q=" + cities[1]

In [65]:
# Create empty lists for city name and today's max temp
city_list = []
max_temps = []

In [70]:
for city in cities[20:50]:
    '''
    Loop through each city in city list to pull today's max temp from OWM API
    Create the query url using api key and city name
    Pull out max temp from JSON response and append to max temp list
    For cities that are not in OWM database, return NA
    To comply with OWM's max 60 calls per minute, delay code 1 second each iteration
    '''
    
    query_url = url + api_key + "&q=" + city
    weather_response = requests.get(query_url)
    weather_json = weather_response.json()
    
    try:
        max_temps.append(weather_json['main']['temp_max'])
    except:
        max_temps.append('NA')
        
    time.sleep(1)
    
    

In [71]:
print(city_list)
print(max_temps)

['nikolskoye', 'illoqqortoormiut', 'NA', 'port alfred', 'samusu', 'NA', 'belaya gora', 'avarua', 'vaini', 'bolshegrivskoye', 'NA', 'punta arenas', 'cherskiy', 'rungata', 'NA', 'maniitsoq', 'longyearbyen', 'alta floresta', 'port macquarie', 'sungaipenuh', 'east london', 'coihaique', 'shingu', 'kupang', 'yeletskiy', 'NA', 'galveston', 'barrow', 'kapaa', 'victoria', 'najran', 'pryazha', 'xichang', 'grenaa', 'te anau']
[59, 'NA', 58.71, 'NA', 55.47, 70.32, 69.15, 'NA', 33.8, 48.18, 'NA', 47.91, 46.4, 85.8, 60.8, 64.83, 65.37, 37.4, 86, 80.6, 'NA', 82.4, 60.69, 84.2, 80.6, 82.4, 51.8, 57.81, 62.6, 38.55, 59, 'NA', 58.71, 'NA', 55.47, 70.32, 69.15, 'NA', 33.8, 48.18, 'NA', 47.91, 46.4, 85.8, 60.8, 64.83, 65.37, 37.4, 86, 80.6, 'NA', 82.4, 60.69, 84.2, 80.6, 82.4, 51.8, 57.81, 62.6, 38.55]


In [68]:
len(city_list)

35

In [72]:
len(max_temps)

60

In [57]:
len(max_temps[10:])

615

In [58]:
max_temps = max_temps[10:]

In [73]:
query_url = url + api_key + "&q=" + cities[38]
print(query_url)
weather_response = requests.get(query_url)
weather_json = weather_response.json()

print(weather_json['main']['temp_max'])

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=e366610da7b576658f8ea67ee1c64590&q=shingu
86


In [44]:
print(cities[38])

georgetown


In [59]:
city_max_temps = pd.DataFrame({'city' : cities, 'max_temp' : max_temps})

In [75]:
print(json.dumps(weather_json, indent = 4, sort_keys = True))

{
    "base": "stations",
    "clouds": {
        "all": 40
    },
    "cod": 200,
    "coord": {
        "lat": 33.72,
        "lon": 135.99
    },
    "dt": 1533600000,
    "id": 1847947,
    "main": {
        "humidity": 66,
        "pressure": 1005,
        "temp": 86,
        "temp_max": 86,
        "temp_min": 86
    },
    "name": "Shingu",
    "sys": {
        "country": "JP",
        "id": 7515,
        "message": 0.0033,
        "sunrise": 1533586316,
        "sunset": 1533635478,
        "type": 1
    },
    "visibility": 10000,
    "weather": [
        {
            "description": "scattered clouds",
            "icon": "03d",
            "id": 802,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 340,
        "speed": 3.36
    }
}
